In [8]:
!git clone https://github.com/jan1na/Neural-Cellular-Automata.git

%cd Neural-Cellular-Automata

Cloning into 'Neural-Cellular-Automata'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 71 (delta 40), reused 35 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 145.30 KiB | 8.07 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/Neural-Cellular-Automata/Neural-Cellular-Automata/Neural-Cellular-Automata


In [ ]:
!pip install -U gdown

file_ids = ['1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj',
            '1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG',
            '1PqrOpn4NJF3xp4Jw2KtRRVD_20Feucsc',
            '1DtZJ5eD2PV4sO51MMQy9CVTZxgBKj_Gx']

file_names = ['pathmnist.npz', 'pathmnist_64.npz', 'pathmnist_128.npz', 'pathmnist_224.npz']

for file_id in file_ids:
    !gdown --id $file_id --output pathmnist.npz

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj
From (redirected): https://drive.google.com/uc?id=1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj&confirm=t&uuid=5c6d6a8e-ea71-4f4c-a9e8-d8eed4f13414
To: /content/Neural-Cellular-Automata/Neural-Cellular-Automata/Neural-Cellular-Automata/pathmnist.npz
100% 206M/206M [00:00<00:00, 247MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG
From (redirected): https://drive.google.com/uc?id=1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG&conf

In [ ]:
!pip install -q medmnist scikit-learn

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from medmnist import PathMNIST
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, mean_absolute_error
import seaborn as sns
from models import NCA, CNNBaseline

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
from google.colab import drive
import os
import shutil

drive_folder = "/content/Neural-Cellular-Automata/"
cache_dir = os.path.expanduser("~/.medmnist")
os.makedirs(cache_dir, exist_ok=True)

resolutions = ["", "_64", "_128", "_224"]
for res in resolutions:
    filename = f"pathmnist{res}.npz"
    src = os.path.join(drive_folder, filename)
    dst = os.path.join(cache_dir, filename)

    if os.path.exists(src):
        shutil.copyfile(src, dst)
        print(f"Copied {filename} to cache.")
    else:
        print(f"File not found in Drive: {filename}")

In [ ]:
nca = NCA().to(device)
nca.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_nca_pathmnist.pth"))
nca.eval()

cnn = CNNBaseline().to(device)
cnn.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_cnn_pathmnist.pth"))
cnn.eval()

In [ ]:
def get_loader(size, batch_size=64):
    transform = transforms.Compose([transforms.ToTensor()])
    dataset = PathMNIST(split="test", size=size, download=False, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
@torch.no_grad()
def evaluate(model, loader, name="Model", size=28, save_dir="/content/drive/MyDrive/NCA/results", is_NCA=False):
    all_preds, all_labels = [], []

    for x, y in loader:
        x, y = x.to(device), y.squeeze()
        if is_NCA:
            out, rgb_steps = model(x, True)
        else:
            out = model(x)
        pred = out.argmax(dim=1).cpu().numpy()
        label = y.numpy()
        all_preds.extend(pred)
        all_labels.extend(label)

    # Metrics
    cm = confusion_matrix(all_labels, all_preds)
    bal_acc = balanced_accuracy_score(all_labels, all_preds)
    mae = mean_absolute_error(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4)

    # Logging
    print(f"\n{name} @ {size}x{size}")
    print("Balanced Accuracy:", f"{bal_acc:.4f}")
    print("Mean Absolute Error (MAE):", f"{mae:.4f}")
    print(report)

    # Save confusion matrix
    os.makedirs(save_dir, exist_ok=True)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix: {name} @ {size}x{size}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    fname = f"{save_dir}/cm_{name.replace(' ', '_')}_{size}x{size}.png"
    plt.savefig(fname)
    plt.close()
    print(f"Confusion matrix saved to: {fname}")

In [ ]:
for size in [28, 64, 128, 224]:
    print(f"\n==============================")
    print(f"Resolution: {size}x{size}")
    loader = get_loader(size)

    print("CNN:")
    evaluate(cnn, loader, name="CNN", size=size)

    print("NCA:")
    evaluate(nca, loader, name="NCA", size=size, is_NCA=True)